# Resnet Approach

## Setup

In [ ]:
! pip install --quiet "ipython[notebook]==7.34.0, <8.17.0" "setuptools>=68.0.0, <68.3.0" "tensorboard" "lightning>=2.0.0" "urllib3" "torch==2.3.1" "matplotlib" "pytorch-lightning>=1.4, <2.1.0" "seaborn" "torchvision" "numpy" "pandas" "tensorflow" "scikit-learn" "torchmetrics>=0.7, <1.3" "matplotlib>=3.0.0, <3.9.0"

## Import Standard Libraries

In [ ]:
import lightning as L
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
from lightning.pytorch.callbacks import LearningRateMonitor, ModelCheckpoint
from torchvision import transforms
import numpy as np

# Setting the seed
L.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

if device == torch.device("cuda:0"):
  print('Using GPU')
else:
  print('GPU is not detected.')

## Download data from Kaggle
This is only needed if you're running on Google Colab. If you're running this notebook locally, ensure that the data is downloaded and in the same folder (i.e. `train.csv`, `test.csv`, `target_name_meta.tsv`, `test_images` and `train_images` should be in the current directory).

Connect to Kaggle via API

In [ ]:
!pip install --quiet kaggle
!echo '{"username":"katsocatso","key":"6bee42e5aa15cc4b7efa6106cc128fa7"}' > kaggle.json
!mkdir -p /root/.kaggle
!mv kaggle.json /root/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

Start Download

In [ ]:
!kaggle competitions download -c cs-480-2024-spring
!unzip -qq *.zip

## Import data into dataset

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd
from PIL import Image
from sklearn.preprocessing import MinMaxScaler
import math

def log10_with_neg(x):
  if x == 0:
    return 0
  elif x < 0:
    return -math.log10(-x)
  else:
    return math.log10(x)


def inverse_log10_with_neg(x):
  if x == 0:
    return 0
  elif x < 0:
    return -(10 ** -x)
  else:
    return 10 ** x

inverse_log10_with_neg_all = np.vectorize(inverse_log10_with_neg)

def preprocess_log_all_targets(df, log_func):
  target_cols = df.columns[-num_targets:]
  for col in target_cols:
    df[col] = df[col].map(log_func)

def remove_ids_from_df(df):
  ids = df['id'].values
  df = df.iloc[:, 1:]
  return ids, df


def normalize(df):
  scaler = MinMaxScaler()
  normalized_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
  return scaler, normalized_df


def remove_outliers(df):
  target_cols = df.columns[-num_targets:]
  for col in target_cols:
    mean = df[col].mean()
    std = df[col].std()
    df = df[(df[col] >= mean - 3 * std) & (df[col] <= mean + 3 * std)]
  return df


num_input_traits = 163
num_targets = 6

# Uncomment when running in Colab
# train_image_path = 'data/train_images'
# train_csv_file = 'data/train.csv'

# test_image_path = 'data/test_images'
# test_csv_file = 'data/test.csv'

# Uncomment when running locally
train_image_path = './train_images'
train_csv_file = './train.csv'

test_image_path = './test_images'
test_csv_file = './test.csv'

df_train_and_val = pd.read_csv(train_csv_file)
df_test = pd.read_csv(test_csv_file)

# Split df_train into train and validation set (ratio 4:1)
df_val = df_train_and_val.sample(frac = 0.25)
df_train = df_train_and_val.drop(df_val.index)

# Get the ids and then remove from the df
ids_train, df_train = remove_ids_from_df(df_train)
ids_val, df_val = remove_ids_from_df(df_val)
ids_test, df_test = remove_ids_from_df(df_test)

# Normalize
scaler_train, normalized_df_train = normalize(df_train)
scaler_val, normalized_df_val = normalize(df_val)
scaler_test, normalized_df_test = normalize(df_test)

# targets_df=pd.read_csv('data/target_name_meta.tsv', sep='\t')
targets_df=pd.read_csv('./target_name_meta.tsv', sep='\t')
target_ids = targets_df['trait_ID']
target_indices_train = [df_train.columns.get_loc(trait + "_mean") for trait in target_ids]
print("The traits we are trying to predict are:")
print(targets_df)

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, image_folder, df, ids, transform=None):
        self.image_folder = image_folder
        self.df = df
        self.ids = ids
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        id = self.ids[idx]
        image = Image.open(self.image_folder + f"/{id}.jpeg")

        input_traits = self.df.iloc[idx, :num_input_traits].values.astype('float')
        targets = self.df.iloc[idx, num_input_traits:].values.astype('float')

        image = self.transform(image)
        # image_batch = image.unsqueeze(0) # create a mini-batch as expected by the model
        # print(image_batch.shape)
        
        return image, torch.tensor(input_traits, dtype=torch.float), torch.tensor(targets, dtype=torch.float), id

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
# input_tensor = preprocess(input_image)
# input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model


# Create the dataset
train_dataset = ImageDataset(train_image_path, normalized_df_train, ids_train, transform)
val_dataset = ImageDataset(train_image_path, normalized_df_val, ids_train, transform)
test_dataset = ImageDataset(test_image_path, normalized_df_test, ids_test, transform)

batch_size = 20
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Define the Model

In [ ]:
def get_custom_resnet():
    resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
    resnet.fc = nn.Identity()
    return resnet


class SpecializedModel(nn.Module):
    def __init__(self, resnet, num_aux_vars, hidden_size, output_size):
        super(SpecializedModel, self).__init__()
        self.resnet = resnet
        self.image_fc = nn.Linear(512, hidden_size)
        
        self.trait_fc1 = nn.Linear(num_aux_vars, hidden_size)
        self.trait_fc2 = nn.Linear(hidden_size, hidden_size // 2)

        self.fc1 = nn.Linear(hidden_size + hidden_size // 2, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size // 2)
        self.fc3 = nn.Linear(hidden_size // 2, output_size)
        
        self.relu = nn.ReLU()
        # self.dropout = nn.Dropout(0.2)


    def forward(self, image, input_traits):
        # Image layers
        image_features = self.resnet(image)
        image_features = self.relu(self.image_fc(image_features))
        
        # Auxiliary variable layers
        trait_features = self.relu(self.trait_fc1(input_traits.float()))
        trait_features = self.relu(self.trait_fc2(trait_features))
        
        # Concatenate image + auxiliary
        combined_input = torch.cat((image_features, trait_features), dim=1)
        fc1 = self.relu(self.fc1(combined_input))
        fc2 = self.relu(self.fc2(fc1))
        output = self.fc3(fc2)
        return output


class ResNetExtendedModel(nn.Module):
    def __init__(self, auxiliary_num_vars, num_targets, auxiliary_idx):
        super(ResNetExtendedModel, self).__init__()
        self.worldclim = SpecializedModel(get_custom_resnet(), auxiliary_num_vars['WORLDCLIM'], 64, 64)
        self.soil = SpecializedModel(get_custom_resnet(), auxiliary_num_vars['SOIL'], 64, 64)
        self.modis = SpecializedModel(get_custom_resnet(), auxiliary_num_vars['MODIS'], 64, 64)
        # self.vod = SpecializedModel(get_custom_resnet(), auxiliary_num_vars['VOD'], 64, 64)
        
        combined_input_size = 64 * 3  # outputs from each specialized model
        self.fc1 = nn.Linear(combined_input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_targets)
        self.relu = nn.ReLU()
        # self.dropout = nn.Dropout(0.2)

        # Save the indices of each category of auxiliary vars
        self.auxiliary_idx = auxiliary_idx

    def forward(self, image, input_traits):
        # Process through each category-specific network
        worldclim_start, worldclim_end = self.auxiliary_idx['WORLDCLIM'][0], self.auxiliary_idx['WORLDCLIM'][1]
        worldclim_output = self.worldclim(image, input_traits[:, worldclim_start:worldclim_end])

        soil_start, soil_end = self.auxiliary_idx['SOIL'][0], self.auxiliary_idx['SOIL'][1]
        soil_output = self.soil(image, input_traits[:, soil_start:soil_end])

        modis_start, modis_end = self.auxiliary_idx['MODIS'][0], self.auxiliary_idx['MODIS'][1]
        modis_output = self.modis(image, input_traits[:, modis_start:modis_end])

        # vod_start, vod_end = self.auxiliary_idx['VOD'][0], self.auxiliary_idx['VOD'][1]
        # vod_output = self.vod(image, input_traits[:, vod_start:vod_end])

        # Concatenate features from all category-specific networks
        combined_input = torch.cat((worldclim_output, soil_output, modis_output), dim=1)

        # Final fully connected layers
        output = self.relu(self.fc1(combined_input))
        output = self.relu(self.fc2(output))
        # x = self.dropout(x)
        output = self.fc3(output)
        return output

## Train

In [ ]:
def train(dataloader, model, criterion, optimizer, device):
    model.train()
    running_loss = 0

    i = 0
    for images, input_traits, targets, _ in dataloader:
        images = images.to(device)
        input_traits = input_traits.to(device)
        targets = targets.to(device)

        if i % 50 == 0 and i != 0:
            print(f"Trained {i} batches so far...")

        optimizer.zero_grad()
        pred = model.forward(images, input_traits)
        # pred = pred.cpu()

        loss = criterion(pred, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        i += 1
    return running_loss


def validate(dataloader, model, criterion, scaler):
    model.eval()
    num_batches = len(dataloader)
    test_loss = 0

    i = 0
    with torch.no_grad():
        for images, input_traits, targets, _ in dataloader:
            images = images.to(device)
            input_traits = input_traits.to(device)
            targets = targets.to(device)

            if i % 50 == 0 and i != 0:
                print(f"Tested {i} batches so far...")
            pred = model.forward(images, input_traits)

            # Move stuff to CPU first so its compatible with the .to('cuda') stuff
            pred_cpu = pred.cpu().numpy()
            targets_cpu = targets.cpu().numpy()

            # Append pred to 163 cols of zeroes, then denormalize
            # We convert from numpy <-> Tensor because inverse_transform only takes np.array
            zeroes = np.zeros((pred_cpu.shape[0], num_input_traits))
            denormalized_pred = scaler.inverse_transform(np.hstack([zeroes, pred_cpu]))[:, -num_targets]
            denormalized_pred = torch.from_numpy(denormalized_pred)
            # denormalized_pred = torch.from_numpy(inverse_log10_with_neg_all(denormalized_pred))
            
            # Do the same thing for targets, then compare
            denormalized_targets = scaler.inverse_transform(np.hstack([zeroes, targets_cpu]))[:, -num_targets]
            denormalized_targets = torch.from_numpy(denormalized_targets)
            # denormalized_targets = torch.from_numpy(inverse_log10_with_neg_all(denormalized_targets))

            test_loss += criterion(denormalized_pred, denormalized_targets).item()
            i += 1

    test_loss /= num_batches
    return test_loss

Find the start and end index of each category in the csv

In [ ]:
id_to_category = {
    0: 'WORLDCLIM',
    1: 'SOIL',
    2: 'MODIS',
    3: 'VOD'
}

auxiliary_idx = {
    'WORLDCLIM': [],
    'SOIL': [],
    'MODIS': [],
    'VOD': []
}

curr_id = 0
for idx, col in enumerate(df_train.columns[:num_input_traits]):
    curr_category = id_to_category[curr_id]

    if not auxiliary_idx[curr_category]:
        # To start the first category
        auxiliary_idx[curr_category].append(idx)
        
    if curr_category not in col:
        auxiliary_idx[curr_category].append(idx)
        curr_id += 1
        # Starting the next category
        auxiliary_idx[id_to_category[curr_id]].append(idx)

# Add the ending idx of the last category
auxiliary_idx[id_to_category[curr_id]].append(idx + 1)

print(auxiliary_idx)

auxiliary_num_vars = {
    'WORLDCLIM': 0,
    'SOIL': 0,
    'MODIS': 0,
    'VOD': 0
}
# Verifying that the aux indices are correct and also initialize auxiliary_num_vars
for id in id_to_category:
    category = id_to_category[id]
    aux_id = auxiliary_idx[category]
    auxiliary_num_vars[category] = aux_id[1] - aux_id[0]

    cols = df_train.columns[aux_id[0]: aux_id[1]].tolist()
    print(id_to_category[id], len(cols))
    print(aux_id[1] - aux_id[0])
    print(cols)
print(auxiliary_num_vars)


Now actually do the training and validating

In [ ]:
import copy
import sys

model = ResNetExtendedModel(auxiliary_num_vars, num_targets, auxiliary_idx)

# best_model = copy.deepcopy(model.state_dict())
model.to('cuda')

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

test_loss = validate(val_loader, model, criterion, scaler_train)

In [ ]:
import copy
import sys

# Freeze layers
# for param in model.parameters():
#      param.requires_grad = False

# for name, param in model.named_parameters():
#     if "layer4" in name:
#         param.requires_grad = True

model = ResNetExtendedModel(auxiliary_num_vars, num_targets, auxiliary_idx)
best_model = copy.deepcopy(model.state_dict())
model.to('cuda')

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# train_accuracy_metrics = []
train_loss_metrics = []
# test_accuracy_metrics = []
test_loss_metrics = []

# Training the model

num_epochs = 5
epochs_lst = range(1, num_epochs + 1)
best_loss = 9223372036854775807

# Implementing early stopping
# patience = 3
# patience_ctr = 0
# tolerance = 0.01

for epoch in epochs_lst:
    print(f"Starting epoch {epoch}")
    train_accuracy = 0

    # Training
    train_loss = train(train_loader, model, criterion, optimizer, device)
    train_loss_metrics.append(train_loss)

    # Testing on validation set
    test_loss = validate(val_loader, model, criterion, scaler_train)
    test_loss_metrics.append(test_loss)
    # test_accuracy_metrics.append(test_accuracy)

    if test_loss < best_loss:
        best_loss = test_loss
        # patience_ctr = 0
        best_model = copy.deepcopy(model.state_dict())
    # else:
    #     patience += 1
    
    # if patience_ctr >= patience:
    #     break

    print(f"test_loss = {test_loss} for epoch {epoch}")
    print(f"Done epoch {epoch}!")
print("Done training and testing!")
model.load_state_dict(best_model)

## Generate Plots

In [ ]:
import matplotlib.pyplot as plt

print("Testing loss")
print(test_loss_metrics)
print()

print("Training loss")
print(train_loss_metrics)
print()

plt.plot(epochs_lst, test_loss_metrics)
plt.title("Test loss vs Epochs")
plt.ylabel("Test loss")
plt.xlabel("Epochs")
plt.show()


## Generate Predictions for Test set

In [ ]:
import numpy as np

def predict(model, dataloader, scaler):
    model.eval()

    predictions = []
    ids = []

    i = 0
    with torch.no_grad():
        for images, input_traits, _, id in dataloader:
            images = images.to(device)
            input_traits = input_traits.to(device)
            # input_traits = input_traits.to(device)
            
            if i % 50 == 0 and i != 0:
                print(f"Tested {i} batches so far...")
            pred = model(images, input_traits)
            pred = pred.cpu().numpy()
            # pred = inverse_log10_with_neg_all(pred)
            predictions.append(pred)
            ids.extend(id.numpy())

            i += 1
    print("Done testing!")

    return ids, predictions

ids, predictions = predict(model, test_loader, scaler_test)

Save to csv

In [ ]:
print(len(ids))
predictions = np.vstack(predictions)

scaled_test_data = normalized_df_test.to_numpy()

# Concatenate original normalized test data with predictions and denormalize
normalized_test_data_with_pred = np.hstack([scaled_test_data, predictions])
denormalized_predictions = scaler_train.inverse_transform(normalized_test_data_with_pred)[:, -num_targets:]

submission_df = pd.DataFrame(denormalized_predictions, columns=[
    'X4','X11','X18','X26','X50','X3112'
])
submission_df.insert(0, 'id', ids)
submission_df.to_csv('submission_resnet_ensemble_3epoch.csv', index=False)
print("Predictions saved to submission_resnet.csv!")